In [1]:
import os
import torch

import utils

%load_ext autoreload
%autoreload 2

CUDA_LAUNCH_BLOCKING=1
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
print(os.listdir("../data/"))


['kawiki.txt', 'data.txt', 'data2.txt']


In [2]:
from utils import GeorgianLanguageDatasetLoader
import gc
gc.collect()
torch.cuda.empty_cache()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataset = GeorgianLanguageDatasetLoader("../data/kawiki.txt", 5, device)

In [3]:
vocab = dataset.get_vocabulary()
print(vocab.get_itos()[:20])

['<unk>', ',', '.', 'და', ')', '(', '—', '“', '„', ':', 'იყო', 'წელს', 'წლის', 'რომელიც', 'რომ', 'საქართველოს', ';', '-', 'ამ', 'მისი']


In [4]:
train_data, valid_data, test_data = dataset.get_data()
train_data[:10]

tensor([  600,    82, 15948,  7157,  3938,     9,  6244,  4313,  3169,  2557])

In [5]:
' '.join([vocab.get_itos()[i] for i in train_data[40:60]])

'რომელიც კონკურენციას უწევს თავის უფრო ადრეულ ანალოგებს Abobe Flex და Microsoft Silverlight . მოგვიანებით ჩნდება ( ალბათ , ქრისტიანობის და'

In [6]:
text_pipeline = dataset.get_text_pipeline()
text_pipeline("შავი კაცი მიდიოდა")

[263, 395, 0]

In [7]:
train_data_batched, val_data_batched = dataset.get_batched_data()

In [8]:
train_data_batched.shape, train_data_batched.device

(torch.Size([5, 17822]), device(type='cuda', index=0))

In [9]:
x, y = utils.get_batch(train_data_batched, 1)

In [10]:
x.shape, y.shape, train_data_batched.shape

(torch.Size([5, 10]), torch.Size([5, 10]), torch.Size([5, 17822]))

In [11]:
x.storage().data_ptr() == train_data_batched.storage().data_ptr()

True

In [12]:
x[0], y[0]

(tensor([   82, 15948,  7157,  3938,     9,  6244,  4313,  3169,  2557,  1889],
        device='cuda:0'),
 tensor([15948,  7157,  3938,     9,  6244,  4313,  3169,  2557,  1889,     1],
        device='cuda:0'))

In [13]:
%reload_ext autoreload

model = utils.LSTMModel(50, 300, len(vocab), device, 1).cuda()
model = model.to(device)
utils.train_loop(model, train_data_batched, batch_size=5)

/home/misho/anaconda3/envs/nlp_final_project/lib/python3.7/site-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.25 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Epoch 100 | Iter 100 | Avg Train Loss 2.1662673690915106 | Dev Perplexity None | LR  0.001
Epoch 200 | Iter 200 | Avg Train Loss 0.03127027733251452 | Dev Perplexity None | LR  0.001
Epoch 300 | Iter 300 | Avg Train Loss 0.01069496932439506 | Dev Perplexity None | LR  0.001
Epoch 400 | Iter 400 | Avg Train Loss 0.006923605049960316 | Dev Perplexity None | LR  0.001
Epoch 500 | Iter 500 | Avg Train Loss 0.005484594712033868 | Dev Perplexity 27861636.931879934 | LR  0.001


In [14]:
text = utils.generate_text(model,device, dataset.vocab, 'იკვლევდა ევროპის და განსაკუთრებით', 20)

In [15]:
print(text)

იკვლევდა ევროპის და განსაკუთრებით . . წლის ქვეშ . და და ბოლომდე ბოლომდე ბოლომდე ბოლომდე და აქვს და აქვს . პორტებია ბოლომდე . მხარის


In [16]:
torch.cuda.empty_cache()

In [17]:
geo_model = utils.GeorgianWord2Vec()
geo_model.train("../data/kawiki.txt")

Initializing data


[nltk_data] Downloading package punkt to /home/misho/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Model created!


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [18]:
embeddings = torch.FloatTensor(geo_model.get_model().wv.vectors)
embeddings.shape

torch.Size([50778, 100])

In [19]:
model = utils.LSTMModel(128, 600, len(vocab), device, num_layers=1, embeddings=embeddings).cuda()
model = model.to(device)
utils.train_loop(model, train_data_batched, batch_size=5)

/home/misho/anaconda3/envs/nlp_final_project/lib/python3.7/site-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.25 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Epoch 100 | Iter 100 | Avg Train Loss 3.014884192943573 | Dev Perplexity None | LR  0.001
Epoch 200 | Iter 200 | Avg Train Loss 0.5420066811144352 | Dev Perplexity None | LR  0.001
Epoch 300 | Iter 300 | Avg Train Loss 0.0977376028522849 | Dev Perplexity None | LR  0.001
Epoch 400 | Iter 400 | Avg Train Loss 0.11779415991157294 | Dev Perplexity None | LR  0.001
Epoch 500 | Iter 500 | Avg Train Loss 0.04499049412086606 | Dev Perplexity 203373217731388.28 | LR  0.001


In [20]:
text = utils.generate_text(model,device, dataset.vocab, 'პოლიტიკური კარიერა', 10)
print(text)

პოლიტიკური კარიერა აღმოჩენილიქნა სქესს სინათლის მიეკუთვნებიან მამრობითი აქვს მამრობითი და გავრცელებისა :
